In [1]:
import pandas as pd
import numpy as np

In [3]:
df= pd.read_csv('/Users/thuyt/Documents/GitHub/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')

In [4]:
df.head()

,uuid,region,country,proj_nm,st_date,end_date,awd_amount,awd_value,implementer,sector,...,rfp_rftop,final_res,mind_eval,final_eval,audit,interim_rep,addl_docs,addl_info,corruption,stability
0,1,South and Central Asia (SCA),Afghanistan,Assistance for Afghanistan's Anti-Corruption A...,2010,2013,e) $25 M-$50 M,26642989,Management Systems International (MSI),"Independent Agencies, Civil Society",...,NaN,http://pdf.usaid.gov/pdf_docs/PA00JP3T.pdf,NaN,NaN,NaN,NaN,http://www.msiworldwide.com/project/supporting...,NaN,High Corruption,Unstable
1,2,South and Central Asia (SCA),Afghanistan,Rule of Law Stabilization Program - Formal Com...,2010,2012,e) $25 M-$50 M,33700000,TetraTech ARD,Rule of Law,...,NaN,NaN,NaN,http://pdf.usaid.gov/pdf_docs/PDACU496.pdf,NaN,NaN,NaN,NaN,Very High Corruption,Very Unstable
2,3,South and Central Asia (SCA),Afghanistan,Afghanistan Rule of Law Stabilization Project ...,2011,2012,c) $5 M-$10 M,10000000,"Checchi and Company Consulting, Inc.",Rule of Law,...,NaN,http://pdf.usaid.gov/pdf_docs/PDACY340.pdf,NaN,http://pdf.usaid.gov/pdf_docs/PDACW029.pdf,NaN,NaN,http://www.checchiconsulting.com/index.php?opt...,NaN,Very High Corruption,Very Unstable
3,4,South and Central Asia (SCA),Afghanistan,Political Party Assessment,2012,2012,a) <$1 M,NaN,"Checchi and Company Consulting, Inc.",Elections and Political Processes,...,NaN,http://pdf.usaid.gov/pdf_docs/PNADZ833.pdf,NaN,NaN,NaN,NaN,NaN,NaN,Very High Corruption,Very Unstable
4,30,Europe and Eurasia (E&E),Albania,Justice Sector Strengthening Project (JuST),2010,2015,c) $5 M-$10 M,9012613,Chemonics,"Rule of Law, Civil Society",...,https://www.fbo.gov/index?s=opportunity&mode=f...,NaN,NaN,NaN,NaN,http://pdf.usaid.gov/pdf_docs/PA00HWM1.pdf,NaN,NaN,High Corruption,Unstable


In [5]:
df.drop(['proj_res','rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1,inplace=True)

In [8]:
df.isna().sum()

uuid                  0
region                0
country               0
proj_nm               0
st_date               0
end_date              0
awd_amount            0
awd_value            54
implementer           0
sector                0
intervention          0
intervention_type     2
proj_desc             0
corruption            0
stability             0
dtype: int64

## Remove punctuation/lower casing

In [9]:
# Load the regular expression library
import re
# Remove punctuation
df['proj_desc'] = df['proj_desc'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df['proj_desc'] = df['proj_desc'].map(lambda x: x.lower())
# Print out the first rows of df
df['proj_desc'].head()

0    the five components of this program seek to ta...
1    the overall goal of the program was to support...
2    the project focused on strengthening and impro...
3    the political party assessment team aimed to r...
4    the justice sector strengthening project (just...
Name: proj_desc, dtype: object

## Tokenize words and further clean-up text

In [10]:
%%time
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data =df.proj_desc.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['the', 'five', 'components', 'of', 'this', 'program', 'seek', 'to', 'take', 'what', 'already', 'exists', 'and', 'improve', 'upon', 'it', 'implement', 'it', 'and', 'find', 'ways', 'to', 'ensure', 'that', 'the', 'progress', 'achieved', 'is', 'sustainable', 'over', 'the', 'long', 'term', 'the', 'project', 'will', 'help', 'the', 'government', 'of', 'the', 'islamic', 'republic', 'of', 'afghanistan', 'strengthen', 'its', 'high', 'office', 'of', 'oversight', 'hoo', 'making', 'it', 'strong', 'effective', 'institution', 'that', 'is', 'able', 'to', 'lead', 'monitor', 'coordinate', 'and', 'report', 'on', 'efforts', 'to', 'combat', 'corruption', 'across', 'the', 'country', 'the', 'project', 'will', 'work', 'and', 'implement', 'program', 'of', 'institutional', 'development', 'and', 'sustainability', 'it', 'will', 'also', 'support', 'the', 'office', 'in', 'carrying', 'out', 'its', 'priority', 'in', 'asset', 'registration', 'and', 'verification', 'complaints', 'management', 'and', 'case', 'tracking

## Phrase Modeling: Bi-grams 

"Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring. Some examples in our example are: ‘back_bumper’, ‘oil_leakage’, ‘maryland_college_park’ etc."

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

## Remove Stopwords, Make Bigrams and Lemmatize

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
import spacy
import en_core_web_sm
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = en_core_web_sm.load()
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

ModuleNotFoundError: No module named 'spacy'

## Data Transformation: Corpus and Dictionary

In [14]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

NameError: name 'data_lemmatized' is not defined

## Build the Base Model

In [100]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

## View the topics in LDA model

In [101]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.033*"improve" + 0.031*"increase" + 0.030*"health" + 0.030*"service" + '
  '0.025*"project" + 0.018*"capacity" + 0.017*"community" + 0.016*"transition" '
  '+ 0.016*"design" + 0.015*"level"'),
 (1,
  '0.043*"project" + 0.021*"support" + 0.021*"capacity" + 0.020*"actor" + '
  '0.018*"critical" + 0.018*"provide" + 0.017*"financial" + 0.017*"relate" + '
  '0.017*"program" + 0.015*"implement"'),
 (2,
  '0.036*"program" + 0.023*"aim" + 0.020*"service_delivery" + 0.020*"document" '
  '+ 0.015*"improve" + 0.014*"contribute" + 0.014*"system" + '
  '0.014*"facilitate" + 0.013*"national" + 0.011*"create"'),
 (3,
  '0.025*"corruption" + 0.025*"program" + 0.018*"support" + 0.017*"improve" + '
  '0.017*"government" + 0.015*"trade" + 0.015*"reduce" + 0.014*"investment" + '
  '0.013*"administrative" + 0.011*"reform"'),
 (4,
  '0.041*"reform" + 0.032*"legal" + 0.029*"project" + 0.025*"health" + '
  '0.023*"sector" + 0.022*"policy" + 0.021*"increase" + 0.021*"palestinian" + '
  '0.020*"support

## Compute Model Perplexity and Coherence Score

In [105]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.44381476392863595


The baseline coherence score is 0.4438

## Hyperparameter Tuning

In [48]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [50]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus,  int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results1.csv', index=False)
    pbar.close()


  0%|          | 0/540 [02:05<?, ?it/s]

100%|██████████| 540/540 [26:17<00:00,  2.92s/it]


In [51]:
pwd

'/Users/yanglingxi/Documents/GitHub/Anti-Corruption1/VIBHU_ROUGH'

In [54]:
pd.DataFrame(model_results).to_csv('lda_tuning_results1.csv', index=False)


In [55]:
df1=pd.read_csv('/Users/yanglingxi/Documents/GitHub/Anti-Corruption1/VIBHU_ROUGH/lda_tuning_results1.csv')


,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.383277
1,75% Corpus,2,0.01,0.31,0.353767
2,75% Corpus,2,0.01,0.61,0.355025
3,75% Corpus,2,0.01,0.9099999999999999,0.342679
4,75% Corpus,2,0.01,symmetric,0.381586
...,...,...,...,...,...
535,100% Corpus,10,asymmetric,0.01,0.443852
536,100% Corpus,10,asymmetric,0.31,0.451774
537,100% Corpus,10,asymmetric,0.61,0.433383
538,100% Corpus,10,asymmetric,0.9099999999999999,0.412109


In [69]:
df1.sort_values('Coherence', ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
533,100% Corpus,10,symmetric,0.9099999999999999,0.535162
323,100% Corpus,3,symmetric,0.9099999999999999,0.481487
398,100% Corpus,6,0.31,0.9099999999999999,0.477724
275,100% Corpus,2,0.31,0.01,0.476939
458,100% Corpus,8,0.31,0.9099999999999999,0.475602
...,...,...,...,...,...
88,75% Corpus,4,asymmetric,0.9099999999999999,0.311957
148,75% Corpus,6,asymmetric,0.9099999999999999,0.311957
118,75% Corpus,5,asymmetric,0.9099999999999999,0.311957
28,75% Corpus,2,asymmetric,0.9099999999999999,0.311957


## Final Model

In [93]:
lda_model_1= gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='symmetric',
                                           eta=0.9099)

In [106]:
# Compute Coherence Score
coherence_model_lda_1 = CoherenceModel(model=lda_model_1, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda_1 = coherence_model_lda_1.get_coherence()
print('\nCoherence Score: ', coherence_lda_1)


Coherence Score:  0.5335635360731606


In [95]:
pprint(lda_model_1.print_topics())
doc_lda_1 = lda_model_1[corpus]

[(0,
  '0.016*"health" + 0.015*"service" + 0.012*"improve" + 0.011*"increase" + '
  '0.008*"level" + 0.008*"community" + 0.006*"child" + 0.005*"access" + '
  '0.004*"quality" + 0.004*"project"'),
 (1,
  '0.006*"project" + 0.005*"actor" + 0.004*"relate" + 0.004*"scheme" + '
  '0.004*"intend" + 0.004*"number" + 0.004*"small" + 0.004*"financial" + '
  '0.004*"critical" + 0.003*"implement"'),
 (2,
  '0.003*"electoral" + 0.001*"election" + 0.001*"launch" + 0.001*"empowerment" '
  '+ 0.001*"morocco" + 0.001*"main" + 0.001*"program" + 0.001*"politically" + '
  '0.001*"fall" + 0.001*"contentious"'),
 (3,
  '0.016*"corruption" + 0.014*"program" + 0.012*"improve" + 0.011*"government" '
  '+ 0.010*"trade" + 0.009*"reduce" + 0.009*"support" + 0.008*"investment" + '
  '0.007*"administrative" + 0.007*"reform"'),
 (4,
  '0.023*"reform" + 0.017*"legal" + 0.016*"health" + 0.015*"project" + '
  '0.013*"palestinian" + 0.012*"sector" + 0.012*"increase" + 0.012*"policy" + '
  '0.011*"support" + 0.010*"impr

## Visualize Topics

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_1, corpus, id2word)
LDAvis_prepared

## USING TF-IDF

In [ ]:
## TOPICS = 30
# minimum_probability=0.0 - total probability = 1
train_vecs = []
for i in range(len(df)):
    top_topics = (
        lda_model_1.get_document_topics(corpus
                                      ,minimum_probability=0.0))
    topic_vec = [top_topics[i][1] for i in range(10)]
    #topic_vec.extend([len(df.iloc[i].proj_desc)])
    train_vecs.append(topic_vec)


In [ ]:
topic_score = pd.DataFrame(train_vecs)
topic_score.head()